In [10]:
import birdnet
m=birdnet.load('acoustic','2.4','tf')
print('loaded', type(m))

loaded <class 'birdnet.acoustic_models.v2_4.model.AcousticModelV2_4'>


In [23]:
predictions = m.predict("data/")
print(predictions)

Microphone

In [27]:
import sounddevice as sd
fs = 44100
sd.default.samplerate = fs
sd.default.channels = 2

In [29]:
duration = 10.5  # seconds
myrecording = sd.rec(int(duration * fs))
sd.wait()

sd.play(myrecording)
sd.wait()

recording <class 'numpy.ndarray'>
